# Демонстрация построения модели LGD
# !!!Выборка с данными для запуска скрипта не предоставляется

## 1. Загрузка библиотек и выборок

In [ ]:
import pandas as pd
import numpy as np
from vtb_scorekit.data import DataSamples
from vtb_scorekit.woe import WOE
from vtb_scorekit.model import LogisticRegressionModel
from vtb_scorekit.cascade import Cascade

### для запуска скрипта необходимо использовать свою выборку 
df = ...
module_features = ...
###
    

## 2. Создание объектов DataSamples для каждого модуля
ds - словарь вида {название модуля: {тип модели: DataSamples()}}

тип модели = 0 для модели полного выздоровления и 1 - для полного убытка

In [ ]:
ds = {}
for module in module_features:
    ds[module] = {}
    for lgd in [0, 1]:
        print(f'\nCreating DataSamples for {module}_{lgd}...')
        ds[module][lgd] = DataSamples(samples={'Train': df[df[module_sample_type[module]] == 'train'],   # выборка для разработки. Задается в виде словаря {название_сэмпла: датафрейм}, может содержать любое кол-во сэмплов
                                               'Test': df[df[module_sample_type[module]] == 'test']},             
                                      target=f'lgd{lgd}',                                           # целевая переменная
                                      features=module_features[module],                             # список переменных. При None берутся все поля числового типа и нечисловые (кроме target, time_column, id_column, weights) с кол-вом уникльных значений меньше min_nunique
                                      cat_columns=None if module=='ZALOG' else [],                  # список категориальных переменных. При None категориальными считаются все переменные с кол-вом уникальных значений меньше min_nunique
                                      id_column='inn',                                              # уникальный в рамках среза айди наблюдения
                                      time_column='year',                                           # дата среза
                                      feature_descriptions=pd.read_excel('data/LGD_description.xlsx', index_col=0), # датафрейм с описанием переменных. Должен содержать индекс с названием переменных и любое кол-во полей с описанием, которые будут подтягиваться в отчеты
                                      result_folder=f'LGD/{module}_{lgd}',                          # папка, в которую будут сохраняться все результаты работы с этим ДатаСэмплом
                                      n_jobs=4,                                                     # кол-во используемых рабочих процессов, при -1 берется число, равное CPU_LIMIT
                                      random_state=0,                                               # сид для генератора случайных чисел, используется во всех остальных методах, где необходимо
                                      samples_split=None,                                             # словарь с параметрами для вызова метода self.samples_split
                                      bootstrap_split={'df': df[df[module_sample_type[module]].isin(['train', 'test'])]}  # словарь с параметрами для вызова метода self.bootstrap_split
                                      )

## 3. Создание каскада моделей
Каскад состоит из двух моделей, каждая из которых в свою очередь содержит четыре модуля. 

Итоговый скор вычисляется кастомной функцией calc_lgd.

In [ ]:
def calc_lgd(scores):   # scores - список [score_0, score_1, ...], где 
                        #          score_i - скор модели i каскада (self.models[i]), имеет тип pd.Series
    pd0 = 1 / (1 + np.exp(-scores[0]))
    pd1 = 1 / (1 + np.exp(-scores[1]))
    return pd1 + 0.361*(1 - pd0 - pd1)
 
cascade = Cascade(models=[                       # список моделей в каскаде. Элементами списка могут быть объекты класса Cascade, LogisticRegressionModel и названия полей отдельных скоров           
                      Cascade(models=[LogisticRegressionModel(ds=ds['FO'][0], name='FO_0'),        # вложенный каскад для модели полного выздоровления lgd0, который в свою очередь состоит из четырх модулей
                                      LogisticRegressionModel(ds=ds['PRAVO'][0], name='PRAVO_0'),
                                      LogisticRegressionModel(ds=ds['KI'][0], name='KI_0'),
                                      LogisticRegressionModel(ds=ds['ZALOG'][0], name='ZALOG_0'),
                                     ],
                              name='lgd0', 
                              ds=ds['main'][0]),
                      Cascade(models=[LogisticRegressionModel(ds=ds['FO'][1], name='FO_1'),        # вложенный каскад для модели полного убытка lgd1
                                      LogisticRegressionModel(ds=ds['PRAVO'][1], name='PRAVO_1'),
                                      LogisticRegressionModel(ds=ds['KI'][1], name='KI_1'),
                                      LogisticRegressionModel(ds=ds['ZALOG'][1], name='ZALOG_1'),
                                     ],
                              name='lgd1',
                              ds=ds['main'][1])
                         ], 
                  integral=calc_lgd,             # функция, вычисляющая интегральный скор каскада по списку скоров входящих в него моделей. При None интегральный скор вычисляется логрегом
                  ds=ds['main'][0],              # ДатаСэмпл, на котором будет рассчитываться интегральный скор 
                  name='LGD'                     # название каскада
                 )

## 4. Автологрег по каскаду
Рекурсивно происходит вызов метода self.auto_logreg() для всех элементов каскада:
1) Если модель в каскаде не имеет биннинга (self.transformer=None), то для нее выполняется автобиннинг. 
2) Если не заданы коэффициенты в self.coefs, то выполняется МФА
3) Генерируется отчет для полученной модели

После обучения всех моделей в каскаде вычисляются их скоры на ДатаСэмпле self.ds (score_0, score_1, ...) и затем по ним вычисляется интегральный скор каскада:
- При self.integral=None строится логрег на скорах score_0, score_1, ...
- При заданном self.integral интегральный скор вычисляется как score_integral = self.integral([df[score_0], df[score_1], ...])

In [ ]:
%%time
cascade.auto_logreg(ds=None,                          # ДатаСэмпл, на котором будет рассчитываться интегральный скор. При None берется self.ds
                    validate=False,                   # флаг для выполнения валидацонных тестов
                    out='auto_cascade.xlsx',          # либо строка с названием эксель файла, либо объект pd.ExcelWriter для сохранения отчета
                    save_model='auto_cascade.json'    # название json файла для сохранения каскада
                   )

## 5. Применение модели
### 5.1. С использованием библиотеки vtb_scorekit

In [ ]:
cascade = Cascade()
cascade.load_model(file_name='LGD/main_0/auto_cascade.json')
cascade.scoring(df,                              # ДатаСэмпл или ДатаФрейм. Возвращается объект того же типа
               score_field='lgd_final',          # поле, в которое должен быть записан посчитанный скор
               pd_field=None,                    # поле, в которое должен быть записан посчитанный PD
               scale_field=None                  # поле, в которое должен быть записан посчитанный грейд
              )

### 5.2. Генерация хардкода для использования модели без сторонних библиотек

In [ ]:
cascade.to_py(file_name='',                      # название питоновского файла, куда должен быть сохранен код
             score_field='lgd_final',            # поле, в которое должен быть записан посчитанный скор
             pd_field=None,                      # поле, в которое должен быть записан посчитанный PD
             scale_field=None                    # поле, в которое должен быть записан посчитанный грейд
            )